In [15]:
from subgrounds import Subgrounds
import polars as pl
import pandas as pd
from subgrounds.schema import TypeRef
from subgrounds.subgraph import SyntheticField
from datetime import datetime, timedelta

sg = Subgrounds()

# Load
univ3_sg = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

fp = univ3_sg.Query.liquidityPools

In [16]:
# usdc/dai pool id - 0x5777d92f208679db4b9778590fa3cab3ac9e2168
qp = fp(
    orderBy='timestamp',
    orderDirection='desc',
    where = {
            "pool_":  {"id": '0x5777d92f208679db4b9778590fa3cab3ac9e2168'},
        }
)

In [17]:
df = sg.query_json([
    # qp._select('timestamp'),
    qp._select('pool')._select('name'),
    qp._select('pool')._select('id'),
    # qp._select('pool')._select('inputTokenBalances'), # get input balances from pool entity
    qp._select('inputTokenBalances')
])

# df = sg.query_json([qp])

KeyError: 'TypeMeta.ObjectMeta.field: no field named timestamp for interface LiquidityPool'

In [11]:
first_key = next(iter(df[0].keys()))
pl_df = pl.from_dicts(df[0][first_key])

In [12]:
# use this logic if column is a struct (rows show up as a dict)
for column in pl_df.columns:
    if isinstance(pl_df[column][0], dict):  
        print(column)
        col_names = pl_df[column][0].keys()
        # rename struct columns
        struct_df = pl_df.select(
            pl.col(column).struct.rename_fields([f'{column}_{c}' for c in col_names])
        )
        struct_df = struct_df.unnest(column)
        # add struct_df columns to pl_df and
        pl_df = pl_df.with_columns(struct_df)
        # drop the pl_df column
        pl_df = pl_df.drop(column)

pool


In [13]:
# use this logic if column is a list (rows show up as pl.Series)
for column in pl_df.columns:
    if isinstance(pl_df[column][0], pl.Series):
        print(column)
        # convert struct to array
        struct_df = pl_df.select([pl.col(column).arr.to_struct()])
        # rename struct fields
        struct_df = struct_df.select(
            pl.col(column).struct.rename_fields([f"{column}_{i}" for i in range(len(struct_df.shape))])
        )
        # unnest struct fields into their own columns
        struct_df = struct_df.unnest(column)
        # add struct_df columns to pl_df and
        pl_df = pl_df.with_columns(struct_df)
        # drop the pl_df column
        pl_df = pl_df.drop(column)

inputTokenBalances


In [14]:
pl_df.head(5)

id,timestamp,pool_name,pool_id,inputTokenBalances_0,inputTokenBalances_1
str,i64,str,str,f64,f64
"""0x5777d92f2086…",1684763963,"""Uniswap V3 Dai…","""0x5777d92f2086…",4.9756e25,5.8052e13
"""0x5777d92f2086…",1684760363,"""Uniswap V3 Dai…","""0x5777d92f2086…",4.9660e25,5.8148e13
"""0x5777d92f2086…",1684756631,"""Uniswap V3 Dai…","""0x5777d92f2086…",4.9752e25,5.8062e13
"""0x5777d92f2086…",1684752659,"""Uniswap V3 Dai…","""0x5777d92f2086…",4.9718e25,5.8104e13
"""0x5777d92f2086…",1684749395,"""Uniswap V3 Dai…","""0x5777d92f2086…",4.9676e25,5.8146e13
